In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import sklearn
import sklearn.feature_extraction.text

#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

In [2]:
df = pandas.read_csv('t\df_10.csv', index_col=0)

In [3]:
#Apply our functions
df['tokenized_text'] = df['AB'].apply(lambda x: lucem_illud_2020.word_tokenize(x))
df['normalized_tokens'] = df['tokenized_text'].apply(lambda x: lucem_illud_2020.normalizeTokens(x))

In [4]:
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, 
                                                                 max_features=1000, min_df=3, 
                                                                 stop_words='english', 
                                                                 norm='l2')
groupsTFVects = ngTFVectorizer.fit_transform(df['AB'])

In [5]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

df['reduced_tokens'] = df['normalized_tokens'].apply(lambda x: dropMissing(x, ngTFVectorizer.vocabulary_.keys()))

In [6]:
dictionary = gensim.corpora.Dictionary(df['reduced_tokens'])

In [7]:
## create a list of tuples containing each token and its count 
## use the first half and save the remainder for testing 
corpus = [dictionary.doc2bow(text) for text in df['reduced_tokens']]

In [8]:
# serialize the corpus as a file and load it 
gensim.corpora.MmCorpus.serialize('abstract.mm', corpus)
abmm = gensim.corpora.MmCorpus('abstract.mm')

In [9]:
# correctly formatted corpus that can be used for topic modeling and induction 
senlda = gensim.models.ldamodel.LdaModel(corpus=abmm, id2word=dictionary, num_topics=20, alpha='auto', eta='auto')

In [10]:
# sen1Bow = dictionary.doc2bow(df['reduced_tokens'][0])
# sen1lda = senlda[sen1Bow]
# print("The topics of the text: {}".format(df['TI'][0]))
# print("are: {}".format(sen1lda))

In [11]:
ldaDF = pandas.DataFrame({
        'titles' : df['TI'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in df['reduced_tokens']]
    })

In [12]:
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

In [13]:
K = senlda.num_topics  # N documents, K topics
topic_labels = ['Topic #{}'.format(k) for k in range(K)]

topicsDict = {}
for topicNum in range(senlda.num_topics):
    topicWords = [w for w, p in senlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12,Topic_13,Topic_14,Topic_15,Topic_16,Topic_17,Topic_18,Topic_19
0,determinant,care,factor,group,ci,factor,determinant,factor,patient,determinant,policy,method,risk,care,intervention,child,work,level,factor,age
1,high,factor,food,determinant,population,determinant,factor,age,determinant,include,patient,result,determinant,determinant,determinant,method,determinant,review,result,determinant
2,low,patient,risk,risk,cohort,result,age,determinant,behavior,result,determinant,high,disease,risk,patient,result,patient,result,child,outcome
3,model,determinant,determinant,result,risk,method,intervention,ci,ci,group,child,prevalence,high,service,use,determinant,intervention,factor,group,support
4,care,child,group,high,factor,age,risk,year,model,risk,factor,age,factor,life,work,family,factor,group,need,method
5,child,result,intervention,year,patient,year,association,analysis,physical,method,level,care,use,disease,public,group,group,patient,care,low
6,school,use,include,patient,determinant,medication,child,result,result,need,community,use,medical,factor,method,care,outcome,high,service,patient
7,risk,community,self,analysis,background,patient,high,high,individual,community,disease,determinant,kidney,community,child,year,review,include,intervention,result
8,factor,life,age,population,high,support,people,method,increase,report,service,patient,method,level,group,conclusion,life,determinant,method,stroke
9,process,level,method,intervention,care,use,self,low,factor,model,include,factor,year,patient,intention,factor,result,analysis,conclusion,year


In [14]:
import pyLDAvis.gensim
lda_model = senlda
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.004242  0.010667       1        1  9.028051
7     -0.016305  0.007473       2        1  8.971561
1      0.007872 -0.015622       3        1  8.654759
8     -0.003064  0.004815       4        1  6.973638
13    -0.016570 -0.019685       5        1  6.237433
11    -0.022788  0.010369       6        1  6.093971
15    -0.005684 -0.001165       7        1  5.763107
18     0.006714 -0.009446       8        1  5.173664
16     0.048086  0.009385       9        1  4.953383
9      0.015771 -0.028892      10        1  4.427332
10     0.010536 -0.021593      11        1  4.364088
2     -0.010434 -0.003661      12        1  4.259704
12    -0.009479  0.019050      13        1  4.173129
17     0.015709  0.035050      14        1  3.848819
0     -0.007963 -0.025995      15        1  3.714998
6      0.006664 -0.020199      16        1  3.713065
5      0.013222  0.014299      17        1  3.608698
14     0.035998 -0.018372      18        1  2.638602
19     0.014849  0.050326      19        1  2.113332
4     -0.078891  0.003195      20        1  1.288658, topic_info=    Category        Freq          Term       Total  loglift  logprob
8    Default  427.000000            ci  427.000000  30.0000  30.0000
170  Default  319.000000          work  319.000000  29.0000  29.0000
175  Default  799.000000          care  799.000000  28.0000  28.0000
40   Default  677.000000       patient  677.000000  27.0000  27.0000
139  Default  473.000000  intervention  473.000000  26.0000  26.0000
..       ...         ...           ...         ...      ...      ...
1    Topic20    7.050674      analysis  540.150208   0.0128  -4.9572
17   Topic20    5.716495        follow  203.585876   0.7788  -5.1670
46   Topic20    6.639710        result  853.421082  -0.5046  -5.0173
10   Topic20    6.179021    conclusion  545.574829  -0.1291  -5.0892
115  Topic20    6.034123      research  412.336273   0.1272  -5.1129

[1275 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
336       1  0.026463  ability
336       2  0.039695  ability
336       3  0.092622  ability
336       4  0.211708  ability
336       5  0.119086  ability
...     ...       ...      ...
429      14  0.100224    youth
429      16  0.085906    youth
429      17  0.028635    youth
429      18  0.014318    youth
429      19  0.014318    youth

[7906 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 2, 9, 14, 12, 16, 19, 17, 10, 11, 3, 13, 18, 1, 7, 6, 15, 20, 5])